# 1.環境構築

## 1.1ライブラリのインストール

In [6]:
# langcahianライブラリのインストール
!pip install langchain langchain-community

# openaiライブラリのインストール
!pip install openai

# lnagchainでopenaiを使うためのライブラリをインストール
!pip install langchain_openai

 #pdfを読み込むためのライブラリをインストールする
!pip install pypdf

# embeddingで利用するライブラリのインストール
!pip install tiktoken

# ローカルマシン上で実行できるchromaベクターデータベースのインストール
!pip install chromadb

# 簡単なGUIを作成できるライブラリのインストール
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.0 MB/s eta 0:00:00


## 1.2ChatGPTのAPIキーの登録

In [7]:
import os

#APIキーの登録
os.environ["OPENAI_API_KEY"] = "hp_v8CJhGZOsPl_QHsF1oW6w9lzvA-B5QGi3sQS_DNKBmDe_TUvLlAg95Y_rDzk4ikTu5_yqB5ja8zrYqq5A_iw"

url = "https://api.openai.iniad.org/api/v1/"

## 2.1PromptとChatModelを利用した英訳

In [10]:
# 必要なライブラリのインポート
## PromptTemplateというlangchainが標準で用意してくれているプロンプトのテンプレートをインポート
from langchain import PromptTemplate

## Chat Modelsを使うときはChatOpenAIをインポート
from langchain.chat_models import ChatOpenAI

## Chainを読み込む
from langchain import LLMChain

# Chat Modelの定義：OpenAIのモデル名と回答のランダム性(temprature)を引数で渡してchatというインスタンスを作成
chat = ChatOpenAI(model_name="gpt-4o-mini", temperature=0, openai_api_base=url)

# プロンプトのテンプレート文章を定義
template = """
あなたは以下の役割を持つアシスタントです：
{role_description}

相談内容：
{sentences_before_check}
"""

# Promptの定義：テンプレート文章に役割と相談内容を組み合わせる
prompt = PromptTemplate(
    input_variables=["role_description", "sentences_before_check"],
    template=template
)

# OpenAIのAPIにこのプロンプトを送信するためのチェーンを作成
chain = LLMChain(llm=chat, prompt=prompt)

# アシスタントの役割と相談内容を設定
role_description = "キャバ嬢っぽくこたえてください"
sentences_before_check = "今日のおすすめのランチは?"

try:
    # チェーンを実行し、結果を取得
    result = chain.run({"role_description": role_description, "sentences_before_check": sentences_before_check})

    # 結果の型を確認して出力
    if isinstance(result, dict):
        if 'text' in result:
            print(result['text'])
        else:
            print("結果に 'text' キーが含まれていません:", result)
    elif isinstance(result, str):
        print(result)
    else:
        print("予期しない結果の型:", type(result), result)

except Exception as e:
    print("エラーが発生しました:", str(e))

<ipython-input-10-4cb30a564d4d>:38: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  result = chain.run({"role_description": role_description, "sentences_before_check": sentences_before_check})


おっけー、今日はちょっと特別なランチにしちゃおうかな♪おすすめは、やっぱりおしゃれなカフェでのサラダボウル！新鮮な野菜に、アボカドやグリルチキンをトッピングして、ヘルシーだけど満足感もバッチリよ♡それに、デザートにはフルーツタルトなんてどう？甘いものは別腹って言うしね♪素敵なランチタイムを楽しんでね！✨


# 3.RAGを実装してみよう


## 3.0.ドキュメントの準備

In [11]:
# 利用するPDFファイルのダウンロード
!pip install gdown
!gdown https://drive.google.com/uc?id=1uKRFzHHOsruuMZp8-Wxg5OrVkPQwtUSH

Downloading...
From: https://drive.google.com/uc?id=1uKRFzHHOsruuMZp8-Wxg5OrVkPQwtUSH
To: /content/security-reference-architecture.pdf
100% 6.23M/6.23M [00:00<00:00, 54.6MB/s]


## 3.1.Document Loaderを使ってドキュメントを読み込む


In [31]:
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

# OpenAI埋め込みモデルのインスタンスを作成
embeddings_model = OpenAIEmbeddings(
    openai_api_base= url
)

# 複数のPDFファイルパスをリストで指定
pdf_paths = [
    "/交通安全対策基本法.pdf",
    "/道路交通法.pdf"
]

all_text = ""
for pdf_path in pdf_paths:
    loader = PyPDFLoader(pdf_path)
    pdf_text = "\n".join([page.page_content for page in loader.load()])
    all_text += pdf_text + "\n"


# Text Splitterの設定
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=20,
    length_function=len,
    add_start_index=True
)

# テキストの分割
documents = text_splitter.create_documents([all_text])

# ベクトル化したテキストをChromaDBに保存する
db = Chroma.from_documents(documents,embeddings_model)

## 3.2.Text Splitterを利用して、読み込んだ文書をチャンク化しよう





In [19]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# PDFからテキストを読み込み、一つの文字列に結合
text = "\n".join([pages.page_content for pages in loader.load()])

# Text Splitterの設定（チャンクサイズとオーバーラップを指定）
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=20,
    length_function=len,
    add_start_index=True
)

# テキストを小さなチャンクに分割
documents = text_splitter.create_documents([text])

# 分割されたテキストの特定のチャンクを表示（例：201番目のチャンク）
print(documents[10])

page_content='1. エンジン１気筒当たりの排気量が 49cc以上1の車両の運転者は ，当該車種に適合 する運転免許証
を所有し ていなければならない。  
2. 運転者は， 車両運転中は常に注意を払い，また車両 を容易かつ迅速に操縦できる楽な位置，特
に支障なく身動きできる位置にい るものとする。  
3. 運転者は ，他の道路使用者および 沿道の住人に対して，無用の 妨害を行ってはならない。  
4. 運転中は， 呼気1リットル当たりアルコール分 0.25mg， または血液1 リットル当たりアルコール
分0.50 mg 以上の酒類の摂取を禁止する 。  
5. 無限軌道式（キャタピラ付き）車両による道路の直接走行は， これを禁止する。 当該車両は ，
ゴムタイヤ式の 別の車両に乗せて輸送しなければならない。  
6. 運転中に ，ハンド・フリー型の補助具を付けていない携帯電話を使用してはならない。  
7.  運転者の視界を遮る ような乗客 ，貨物，またはその他の 物品を積載してはならない。  
8. １台の自動二輪車には ，大人２名と子供１名まで乗ることができる。自動 二輪車の 運転者，相' metadata={'start_index': 4941}


In [24]:
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# OpenAI埋め込みモデルのインスタンスを作成
embeddings_model = OpenAIEmbeddings(
    openai_api_base= url
)


# PDFファイルの読み込み
pdf_paths = [
    "/交通安全対策基本法.pdf",
    "/道路交通法.pdf"
]
all_text = ""
for pdf_path in pdf_paths:
    loader = PyPDFLoader(pdf_path)
    pdf_text = "\n".join([page.page_content for page in loader.load()])
    all_text += pdf_text + "\n"


# Text Splitterの設定
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=20,
    length_function=len,
    add_start_index=True
)

# テキストの分割
documents = text_splitter.create_documents([text])

# ベクトル化したテキストをChromaDBに保存する
db = Chroma.from_documents(documents,embeddings_model)

In [29]:
query = "交通事故"
embedding_vector = OpenAIEmbeddings(openai_api_base= url).embed_query(query)
docs = db.similarity_search(query)
print(docs)

[Document(metadata={'start_index': 35250}, page_content='元機関または交通警察に 通報する か，または被害者 を最寄りの病院に連れていくための緊急措置\nを講じるものとする。  \n \n第38条 \n 交通事故 の当事者のいずれか一方 が外国大使または外交官である場合， 当該当事者が， 道路交\n通警察官 と協力して， 事件の検証 および適切な 措置の提案を行う代理人を手配できるように ，交\n通警察官 は，交通事故事件簿を作成し ，当該事故 の報告書を 外務省および 内務省に提出するもの\nとする。  \n \n \n第７章 \n車両および 運輸管理  \n \n第39条 \n 全車種の車両運転者の国内・国際 運転免許証 について は，公共事業運輸省が交付するもの とす'), Document(metadata={'start_index': 35250}, page_content='元機関または交通警察に 通報する か，または被害者 を最寄りの病院に連れていくための緊急措置\nを講じるものとする。  \n \n第38条 \n 交通事故 の当事者のいずれか一方 が外国大使または外交官である場合， 当該当事者が， 道路交\n通警察官 と協力して， 事件の検証 および適切な 措置の提案を行う代理人を手配できるように ，交\n通警察官 は，交通事故事件簿を作成し ，当該事故 の報告書を 外務省および 内務省に提出するもの\nとする。  \n \n \n第７章 \n車両および 運輸管理  \n \n第39条 \n 全車種の車両運転者の国内・国際 運転免許証 について は，公共事業運輸省が交付するもの とす'), Document(metadata={'start_index': 23721}, page_content='元機関または交通警察に 通報する か，または被害者 を最寄りの病院に連れていくための緊急措置\nを講じるものとする。  \n \n第38条 \n 交通事故 の当事者のいずれか一方 が外国大使または外交官である場合， 当該当事者が， 道路交\n通警察官 と協力して， 事件の検証 および適切な 措置の提案を行う代理人を手配できるように ，交\n通警察官 は，交通事故事件簿を作成し ，当該事故 の報告

In [28]:
import gradio as gr
from langchain_core.runnables import RunnablePassthrough
from langchain.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# ドキュメントの読み込みとベクトルストアの作成
retriever = db.as_retriever(search_kwargs={"k": 3})

# テンプレートとプロンプトの定義
template_with_context = """以下のcontextのみに基づいて質問にできるだけ詳しく箇条書きで答えなさい。:
{context}
質問: {question}
"""

prompt_with_context = ChatPromptTemplate.from_template(template_with_context)

# LLMの定義（ストリーミング対応）
llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0,
    openai_api_base= url,
    verbose=True)

def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

# Chainの定義
chain_with_context = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_with_context
    | llm
    | StrOutputParser()
)

def process_query_with_rag(query):
    # 関連ドキュメントの取得
    relevant_docs = retriever.get_relevant_documents(query)
    sources = [doc.page_content for doc in relevant_docs]

    # 回答の生成（ストリーミング）
    answer = ""
    for chunk in chain_with_context.stream(query):
        answer += chunk
        yield answer, "\n\n".join(sources)

# Gradioインターフェースの定義
with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            gr.Markdown("## RAGあり")
            input_text_with_rag = gr.Textbox(lines=2, placeholder="質問を入力してください...")
            output_text_with_rag = gr.Markdown(label="回答")
            sources_with_rag = gr.Textbox(label="参照ソース")
            rag_button = gr.Button("RAGありで質問する")
            rag_button.click(fn=process_query_with_rag, inputs=input_text_with_rag, outputs=[output_text_with_rag, sources_with_rag])

demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://25c6669fc13f82000c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [13]:
import os
## Chat Modelsを使うときはChatOpenAIをインポート

from langchain.document_loaders import PyPDFLoader, PDFMinerLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
import pdfminer

os.environ["OPENAI_API_KEY"] = "hp_v8CJhGZOsPl_QHsF1oW6w9lzvA-B5QGi3sQS_DNKBmDe_TUvLlAg95Y_rDzk4ikTu5_yqB5ja8zrYqq5A_iw"

url = "https://api.openai.iniad.org/api/v1/"
# OpenAI埋め込みモデルのインスタンスを作成
embeddings_model = OpenAIEmbeddings(
    openai_api_base= url
)

# 複数のPDFファイルパスをリストで指定
pdf_paths = [f for f in [
    "Legal_Support_Mate/pdf/パワハラ1.pdf",
    "Legal_Support_Mate/pdf/パワハラ2.pdf",
    "Legal_Support_Mate/pdf/パワハラ3.pdf",
    "Legal_Support_Mate/pdf/パワハラ4.pdf",
    "Legal_Support_Mate/pdf/消費者法1.pdf",
    "Legal_Support_Mate/pdf/消費者法2.pdf",
    "Legal_Support_Mate/pdf/相続権1.pdf",
    "Legal_Support_Mate/pdf/相続権2.pdf",
    "Legal_Support_Mate/pdf/相続権3.pdf",
    "Legal_Support_Mate/pdf/道路交通1.pdf",
    "Legal_Support_Mate/pdf/道路交通2.pdf",
    "Legal_Support_Mate/pdf/道路交通3.pdf",
    "Legal_Support_Mate/pdf/道路交通4.pdf",
    "Legal_Support_Mate/pdf/道路交通5.pdf",
    # HACK!
    # 著作権のPDFファイルはサイズがほかより12倍大きい
    # そのせいか、60分以上かかるので、一旦コメントアウト
    # "Legal_Support_Mate/pdf/著作権.pdf",
    "Legal_Support_Mate/pdf/誹謗中傷1.pdf",
    "Legal_Support_Mate/pdf/誹謗中傷2.pdf",
    "Legal_Support_Mate/pdf/誹謗中傷3.pdf",
    "Legal_Support_Mate/pdf/離婚1.pdf",
    "Legal_Support_Mate/pdf/離婚2.pdf",
    "Legal_Support_Mate/pdf/労働1.pdf",
    ]
]

all_text = []
for pdf_path in pdf_paths:
    loader = PDFMinerLoader(pdf_path)
    pdf_text = "\n".join([page.page_content for page in loader.load()])
    all_text.append(pdf_text)
    print(len(pdf_text), pdf_path)
all_text = "\n".join(all_text)

# Text Splitterの設定
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=20,
    length_function=len,
    add_start_index=True
)

# テキストの分割
documents = text_splitter.create_documents([all_text])

db = Chroma(persist_directory="DB")
# ベクトル化したテキストをChromaDBに保存する
db = Chroma.from_documents(documents,
                            embeddings_model,
                            persist_directory="DB")
# データベースを永続に保存
db.persist()
# 本当は DB/ はgitignoreに入れるべきだが、計算時間がかかるのでアップロードする
db._collection.count()

The PDF <_io.BufferedReader name='Legal_Support_Mate/pdf/パワハラ1.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case
The PDF <_io.BufferedReader name='Legal_Support_Mate/pdf/パワハラ2.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


67773 Legal_Support_Mate/pdf/パワハラ1.pdf
67773 Legal_Support_Mate/pdf/パワハラ2.pdf


Ignoring (part of) ToUnicode map because the PDF data does not conform to the format. This could result in (cid) values in the output. The start object is not a byte.


116834 Legal_Support_Mate/pdf/パワハラ3.pdf
12543 Legal_Support_Mate/pdf/パワハラ4.pdf
128817 Legal_Support_Mate/pdf/消費者法1.pdf
40199 Legal_Support_Mate/pdf/消費者法2.pdf
10874 Legal_Support_Mate/pdf/相続権1.pdf
38576 Legal_Support_Mate/pdf/相続権2.pdf
236115 Legal_Support_Mate/pdf/相続権3.pdf
49593 Legal_Support_Mate/pdf/道路交通1.pdf
40677 Legal_Support_Mate/pdf/道路交通2.pdf
2620 Legal_Support_Mate/pdf/道路交通3.pdf
2361 Legal_Support_Mate/pdf/道路交通4.pdf
11672 Legal_Support_Mate/pdf/道路交通5.pdf
13622 Legal_Support_Mate/pdf/誹謗中傷1.pdf
13853 Legal_Support_Mate/pdf/誹謗中傷2.pdf
236115 Legal_Support_Mate/pdf/誹謗中傷3.pdf
126337 Legal_Support_Mate/pdf/離婚1.pdf
236115 Legal_Support_Mate/pdf/離婚2.pdf
37279 Legal_Support_Mate/pdf/労働1.pdf


4352